<a href="https://colab.research.google.com/github/jerekyyro/Faffe/blob/main/FaFFe_paikannimet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Paikannimien tunnistaminen osumaympäristöstä

Tätä Google Colab -notebookia voi käyttää paikannimien tunnistamiseen xlsx-tiedostossa olevasta tekstistä. Notebook sisältää tekstisoluja (kuten tämä osio), suoritettavia Python-koodisoluja (alla oleva harmaana näkyvä solu) sekä koodin tuottamia tulosteita (muodostuvat koodisolujen alle kun niitä suoritetaan).

**Ennen kuin aloitat**, tallenna itsellesi kopio tästä omaan Google Driveen: *File > Save a copy in Drive*.

Tätä notebookia käytetään suorittamalla yksi kerrallaan koodisoluja. Koodi käynnistetään koodisolun vasemmassa laidassa olevasta play-nappulan näkyisestä nappulasta, joka ilmestyy, kun klikkaat koodisolua tai siirrät kursorin hakasulkeissa olevan numeron kohdalle (esim. [1]).

Jos aktivoit tällaisen tekstisolun, pääset pois tuplaklikkaamalla oikealla tai painamalla SHIFT + ENTER. Voit liikkua solujen välillä nuolinäppäimillä.

SHIFT + ENTER:illä voit myös suorittaa aktiivisen *koodisolun*.

Yhden koodisolun suoritus saattaa kestää jopa minuutin, ehkä enemmän.

Jokaisen koodisolun kohdalla kerrotaan, mitä se tekee ja on lisäohjeita.

# 1. Asennukset.

Alla oleva koodi suorittaa erinäisiä asennuksia. Suorita se aina aloittaessasi istunnon. Jos notebook on käyttämättä jonkin aikaa, istunto katkeaa ja nuo asennukset pitää ajaa uudestaan, jotta alemmat koodisolut toimivat.

In [ ]:
#ASENNUKSET

!pip install -U pip setuptools wheel
!pip install -U spacy

!python -m spacy download fi_core_news_lg
!python -m spacy download sv_core_news_sm

import spacy


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2023-06-02 09:57:05.808675: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.8/230.8 MB 2.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fi_core_news_lg')
2023-06-02 09:57:37.503053: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 50.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('sv_core_

# 2. Kielen valinta.

Suorita JOKO solu #SUOMI tai #RUOTSI sen mukaan, minkä kielistä aineistoa olet syöttämässä.

In [ ]:
#SUOMI
nlp = spacy.load('fi_core_news_lg')


In [ ]:
#RUOTSI
nlp = spacy.load("sv_core_news_sm")


2023-06-02 08:03:54.139301: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 16.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('sv_core_news_sm')


# 3. Tiedostojen lataaminen läpikäytäväksi.

Kun suoritat allaolevan koodin, voit ladata koneeltasi yhden tai useamman xlsx-tiedoston, jotka haluat käydä läpi. Suoritettuasi koodin, paina *Choose files* -nappia.

(Melko hidas!)

In [ ]:
from google.colab import files

uploaded = files.upload()

for file in uploaded:
  print(file)

Saving bindings-punarinta-1.xlsx to bindings-punarinta-1.xlsx
bindings-punarinta-1.xlsx


# 3b. Xlsx-tiedoston aloitusrivi ja muut määritykset.

Alla on oletuksena otsikot sisältäväksi riviksi asetettu 1, mikä tarkoittaa, että otsikot ovat excel-tiedoston toisella rivillä (rivi 1 -> 0, rivi 2 -> 1, jne.), koska digiarkiston tuottamien excelien ensimmäisellä rivillä on url-osoite, jota emme tarvitse.

Muuta siis allaolevassa koodissa olevaa numeroa tarpeen mukaan.

Huomaa, että otsikkoriviä ennen olevat rivit jäävät pois läpikäydystä excelistä!

Voit myös muuttaa läpikäytävän sarakkeen otsikkoa ja välilehden nimeä. Muista suorittaa solu ennen kuin menet eteenpäin.

In [ ]:
aloitusrivi = 1

tekstisarake = "Osumaympäristö"

valilehti = "Tulokset"


# 4. Tiedostojen läpikäynti.

Alla oleva koodi käy läpi Osumaympäristö -sarakkeen ja lisää uuden paikat -sarakkeen, jossa on listattu kunkin rivin osumaympäristön paikat (GPE: geopolitical entity ja LOC: muu lokaatio). Löydetyt paikat tulostuvat alle. Tiedostot käydään läpi yksi kerrallaan, ja kun tiedostot ovat valmiit, ne latautuvat takaisin koneellesi.


In [ ]:
import pandas as pd

def process_line(line):
  doc = nlp(line)
  #entiteetit = ""
  paikat = ""
  for ent in doc.ents:
    #entiteetit += ent.text + " " + ent.label_
    if ent.label_ == "LOC" or ent.label_ == "GPE":
      paikat += ent.text + " " + ent.label_ + " "
  print(paikat)
  return paikat

def run(filename):
  data = pd.read_excel(filename, sheet_name = valilehti, header = aloitusrivi)
  #print(data.head())
  data["Paikat"] = data[tekstisarake].apply(process_line)

  filename = "paikat_"+filename
  data.to_excel(filename)
  #print(data.head())

done = []
for file in uploaded:
  run(file)
  done.append("paikat_"+file)


for new_file in done:
  files.download(new_file)

Suomessa GPE 
Oulun GPE Lapissa LOC Muonion LOC 


Punarinnan GPE Satakunnan GPE Suomessa GPE 
Euroopassa LOC Suomessa GPE 
J*£ GPE Punarinnan GPE Suomessa GPE Suomen GPE 

Suomessakin GPE 
Punarinnan GPE 
Keski-Euroopassa GPE Keski-Euroopassa GPE Britanniaa GPE Saksassa GPE punarinnan GPE Britanniassa GPE Punarinnalle LOC 
Keski-Euroopassa GPE Britteinsaarilla GPE 
Punarinnan GPE punarinnan GPE Punarinnan GPE 
Punarinnalle LOC Joutsenon GPE 
pohjolassa GPE 
Taipalsaarelta GPE Länsi-Saimaan GPE 


Suomessa GPE Suomen GPE Englannissa GPE 
Ylä-Satakunnassa GPE Ylä-Satakunnassa GPE 
Koillismaalla LOC Punarinnalle LOC 
Mäntyharjulla GPE Pertunmaalla GPE Etelä-Savossa GPE 
Punarinnan GPE Puumalassa GPE Puumalassa GPE Puumalassa GPE 
Loviisa GPE Punarinnan GPE Loviisan GPE Punarinnalle LOC 
Punarinnalla GPE Punarinnan GPE nnnila LOC Punarinnalla GPE 
Lontoon GPE Westminster Abbeyssä GPE Lontoossa GPE 
Somerolla GPE Tammelassa GPE 
Punarinnan GPE Punarinnan GPE 
punarinnan GPE Suomeen GPE 
Pu

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#5. Lopetus

Voit tarkastella Colabiin lataamiasi tiedostoja vasemmalla olevan kansio-kuvakkeen takaa. Kun olet ladatut läpikäydyt tiedostot, kannattaa poistaa tiedostot kaikki xlsx-tiedostot.